In [ ]:
!{'python3 -m pip install -U dataflows'}

In [27]:
import requests, time
from dataflows import Flow, checkpoint, printer

PAGE_URL_TEMPLATE = 'http://www.museumsinisrael.gov.il/_Layouts/15/Tmit.SP2013.Mozionim.UI/GetSearch.ashx?type=items&Culture=he-IL&refiners=HebArtDomainName%3A%22%D7%90%D7%9E%D7%A0%D7%95%D7%AA%22&page={page}'
ITEM_URL_TEMPLATE = 'http://www.museumsinisrael.gov.il/he/items/Pages/ItemCard.aspx?IdItem={link}'

session = requests.Session()

def get_page_items(page, retry_num=0):
    if retry_num > 10:
        raise Exception('too many retries for page {}'.format(page))
    url = PAGE_URL_TEMPLATE.format(page=page)
    try:
        res = session.get(url).json()
    except Exception:
        time.sleep(15)
        print('page {} retry_num {}'.format(page, retry_num+1))
        return get_page_items(page, retry_num+1)
    return res

def get_all_page_items():
    total_found = 0
    num_yielded = 0
    for page in range(0,800):
        print(page)
        res = get_page_items(page)
        assert len(res) == 1
        assert res[0]['Type'] == 'פריטים'
        total_found = res[0]['TotalFound']
        cur_page_num_yielded = 0
        for item in res[0]['items']:
            row = {str(k): str(v) for k, v in item.items()}
            row['item_url'] = ITEM_URL_TEMPLATE.format(link=row.pop('Link'))
            yield row
            num_yielded += 1
            cur_page_num_yielded += 1
        print(num_yielded)
        if cur_page_num_yielded < 1:
            break
        time.sleep(2)

Flow(
    get_all_page_items(),
    checkpoint('all_page_items'),
    printer(tablefmt='html', num_rows=1)
).process()[1]

saving checkpoint to: .checkpoints/all_page_items
0


30
1
60
2
90
3
page 3 retry_num 1
120
4
150
5
180
6
210
7
240
8
270
9
300
10
330
11
360
12
390
13
420
14
450
15
480
16
510
17
540
18
570
19
600
20
630
21
660
22
690
23
720
24
750
25
780
26
810
27
840
28
870
29
900
30
930
31
960
32
990
33
1020
34
1050
35
1080
36
1110
37
1140
38
1170
39
1200
40
1230
41
1260
42
1290
43
1320
44
1350
45
1380
46
1410
47
1440
48
1470
49
1500
50
1530
51
1560
52
1590
53
1620
54
1650
55
1680
56
1710
57
1740
58
1770
59
1800
60
1830
61
1860
62
1890
63
1920
64
1950
65
1980
66
2010
67
2040
68
2070
69
2100
70
2130
71
2160
72
2190
73
2220
74
2250
75
2280
76
2310
77
2340
78
2370
79
2400
80
2430
81
2460
82
2490
83
2520
84
2550
85
2580
86
2610
87
2640
88
2670
89
2700
90
2730
91
2760
92
2790
93
2820
94
2850
95
2880
96
2910
97
2940
98
2970
99
3000
100
3030
101
3060
102
3090
103
3120
104
3150
105
3180
106
3210
107
3240
108
3270
109
3300
110
3330
111
3360
112
3390
113
3420
114
3450
115
3480
116
3510
117
3540
118
3570
119
3600
120
3630
121
3660
122
3690
123
3720
124
3750
125


#,Title (string),ThumbImageMono (string),ThumbImage (string),SecendRow (string),therdRow (string),item_url (string)
1,דמות,None,TID157933_ITEM_MAIN_PIC_786479.jpg,"Ora Lahav-Chaaltiel ,","מוזאון ינקו-דאדא, עין הוד",http://www.museumsinisrael.gov.il/he/items/Pages/ItemCard.aspx?IdItem=ICMS-JCO-C.1103
2,עינויי ברתולומאו הקדוש,None,TID60498_ITEM_MAIN_PIC_797974.jpg,"ריברה, ג'וספה דה, ספרדי, פעל בנפולי, 1591–1652 ,","מוזאון ישראל, ירושלים",http://www.museumsinisrael.gov.il/he/items/Pages/ItemCard.aspx?IdItem=ICMS_IMJ_202058
...,,,,,,
23812,עירום נשי,None,TID128411_ITEM_MAIN_PIC_1106257.jpg,"שטרן, פרידל, ישראלית (ילידת גרמניה), 2006-1917 ,","המוזיאון הישראלי לקריקטורה ולקומיקס, חולון",http://www.museumsinisrael.gov.il/he/items/Pages/ItemCard.aspx?IdItem=ICMS-CAR-1003966


checkpoint saved: all_page_items


{'count_of_rows': 23812,
 'bytes': 6482411,
 'hash': '74a90648a2e201581806bed65ebfd208',
 'dataset_name': None}